In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
# importing dataset folder

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.listdir('/content/drive/My Drive')

['Classroom',
 'SWOT Analysis.gdoc',
 'swot analysis (sunshine buddy).pdf',
 'Guitar actual (2).par',
 'guitar CAD (2).png',
 'guitar CAD (1).png',
 'guitar CAD.png',
 'Guitar actual (1).par',
 'Guitar actual.par',
 'Untitled form.gform',
 'IMG_3400.jpg',
 'Complex_4.gdoc',
 'Buddy report.docx',
 'WhatsApp Image 2023-01-08 at 12.50.19.jpeg',
 'ES22BTECH11014_Assig1.pdf',
 'Yashmitha Sunshine',
 'IMG-20230310-WA0021.jpg',
 'IMG-20230320-WA0028.jpg',
 'ID card Mar 21, 2023_1.jpg',
 '16793907569285353538747914663683.jpg',
 'IMG_20230321_150026.jpg',
 'SlotA-students.gsheet',
 'MA1140_ assignment_1-ES22BTECH11014.pdf',
 'IMG-20230408-WA0079.jpg',
 'A1_ES22BTECH11014_2.par',
 'A1_ES22BTECH11014_3.par',
 'A1_ES22BTECH11014_1.par',
 'Milan 2022-23',
 'Screenshot_2023-08-15-17-38-38-54_49b96b5fbae0d12a18edc4a3afe0dfd9.jpg',
 'Screenshot_2023-08-15-19-42-10-71_49b96b5fbae0d12a18edc4a3afe0dfd9.jpg',
 'IMG20230918203856.jpg',
 'IMG20230918204056.jpg',
 'IMG20230918204305.jpg',
 'Folk theatre',
 '

In [ ]:
os.chdir('/content/drive/My Drive/indoorCVPR_09')

In [ ]:
cwd = os.getcwd()
print(cwd)

/content/drive/My Drive/indoorCVPR_09


In [ ]:
print(os.listdir())

['Images']


In [ ]:
#import necessary libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets  import ImageFolder
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd
import os

#define tranformations
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#create the ImageFolder Dataset
dataset = ImageFolder(root='/content/drive/MyDrive/indoorCVPR_09/Images', transform = transform)
classes = dataset.classes

#get all class labels
targets =[sample[1] for sample in dataset.samples]

#first stratified split to create a train+val and test sets
train_val_indices, test_indices = train_test_split(
    np.arange(len(targets)),
    test_size = 0.2,
    stratify = targets,
    random_state = 42
)

#get class labels for the train+val set
train_val_targets = [targets[i] for i in train_val_indices]

#second stratified split to create train and val sets
train_indices, val_indices = train_test_split(
    train_val_indices,
    test_size = 0.25, #0.25*0.8 = 0.2 i.e. 20% of total dataset
    stratify = train_val_targets,
    random_state = 42
)

# create subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
import timm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#load a pretrained ViT model
#model = timm.create_model('resnet50', pretrained = True)
model = timm.create_model('vit_base_patch16_224', pretrained=True)
#model.fc = nn.Linear(model.fc.in_features, 66)
outputs_attrs = 67
num_inputs = model.head.in_features
last_layer = nn.Linear(num_inputs, outputs_attrs)
model.head = last_layer
# Freeze all layers except the head
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the head layer
for param in model.head.parameters():
    param.requires_grad = True

#move the model to appropriate device
model = model.to(device)
print(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

#defining loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# Define the learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
#training loop
def main():
  num_epochs = 10
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      for i, (images, labels) in enumerate(train_loader):
          images, labels = images.to(device), labels.to(device)

          #forward pass
          output = model(images)
          loss = criterion(output, labels)

          #Backward pass and optimisation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          running_loss+=loss.item()
          if (i+1) % 10 == 0:  # Print every 10 batches
              print(f'Batch [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
      # Validation
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      with torch.no_grad():
          for images, labels in val_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              loss = criterion(outputs, labels)
              val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')
    # Test
  model.eval()
  test_loss = 0.0
  correct = 0
  total = 0
  with torch.no_grad():
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {100 * correct / total:.2f}%')


In [ ]:
main()

Batch [10/293], Loss: 4.3177
Batch [20/293], Loss: 3.9674
Batch [30/293], Loss: 3.7886
Batch [40/293], Loss: 4.0049
Batch [50/293], Loss: 3.4791
Batch [60/293], Loss: 3.4600
Batch [70/293], Loss: 3.4369
Batch [80/293], Loss: 3.0931
Batch [90/293], Loss: 3.0729
Batch [100/293], Loss: 2.8908
Batch [110/293], Loss: 2.5537
Batch [120/293], Loss: 2.5110
Batch [130/293], Loss: 2.1033
Batch [140/293], Loss: 2.1320


In [ ]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load your pre-trained model

model.eval()

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Function to make predictions
def predict(image):
    # Transform the input image
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Send the image to the model and get predictions
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]

    return predicted_class

# Define the Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.Image(shape=(224, 224)),
    outputs="text",
    title="Image Classification",
    description="Upload an image to get the classification result."
)

# Launch the Gradio interface
iface.launch()


NameError: name 'model' is not defined